<h1 style="text-align: center;"> Model Tuning </p>

## Notebook Description

In this notebook, time-series data are modeled for forecasting oil stock performance as part of the requirements of the RMDS 2021 Data Science Competition.

##  Table of contents
1. [Required Libraries](#Required-Libraries)
2. [Load Data](#Load-Data)
3. [Format Column Names](#Format-Column-Names)
4. [Set Timestamp Range & Frequency](#Set-Timestamp-Range-&-Frequency)
5. [Merge Data](#Merge-DataFrames)
6. 
7. 
8. 
9. 
10. 
11. [ARIMA Modeling](#ARIMA-Modeling)
12. [Conclusion](#Conclusion)

## Required Libraries

[[ go back to the top ]](#Table-of-contents)

This notebook uses several Python libraries such as:

In [159]:
# Load required packages 
import datetime
from datetime import timedelta
import numpy as np
import pandas as pd

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Time-Series
import statsmodels.api as sm
#from statsmodels.tsa.stattools import adfuller
#from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.stattools import acf, pacf
#from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA, ARIMA
from scipy import signal
import scipy.stats as stats

import warnings
warnings.filterwarnings("ignore")

<a id='Load-Data'></a>

---
## Load Data

[[ go back to the top ]](#Table-of-contents)

In [160]:
# Load Data Function
def LOAD_DATA(filepath, filename):
    # Read CSV files
    if filename.endswith('.csv'):
        new_df = pd.read_csv(filepath+filename)

    # Read Excel files
    elif filename.endswith('.xlsx'):
        new_df = pd.read_excel(filepath+filename)
    print(type(new_df.index))
    if type(new_df.index) != pd.core.indexes.datetimes.DatetimeIndex:
        for col in new_df.columns:
            if col.lower().find('date') != -1:
                print(f"TIMESTAMP FOUND! '{col}'")
                print()
                new_df['date'] = pd.to_datetime(new_df[col]) # format = '%Y/%m/%d'
                new_df.set_index('date', inplace = True)
                # If datetime col was already == 'date', no need to drop col after set_index, otherwise...
                if col != 'date':
                    new_df.drop(columns = col, inplace = True)
                
    # Try to identify the date column
    elif type(new_df.index) == pd.core.indexes.datetimes.DatetimeIndex:
        print('Index already in datetime')
        
    display(new_df.info())
    return new_df

In [161]:
# Load International Sentiment Data

fpath = '../../data/News_AI_Sentiments/'
fname = 'daily-news-sentiment-international.csv'

sentiment_int = LOAD_DATA(filepath = fpath, filename = fname)
#sentiment_int

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'DateTime'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 395 entries, 2000-04-17 to 2021-02-27
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Daily News Sentiment  395 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


None

In [162]:
# Load North American Sentiment Data

fpath = '../../data/News_AI_Sentiments/'
fname = 'daily-news-sentiment-NA.csv'

sentiment_na = LOAD_DATA(filepath = fpath, filename = fname)
#sentiment_na

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'DateTime'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 396 entries, 2013-07-26 to 2021-02-27
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Daily News Sentiment  396 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


None

In [163]:
# Load Stock-Closing-Price by Company Data
fpath = '../../'
fname = 'closing_price_by_company.csv'
closing_price = LOAD_DATA(filepath = fpath, filename = fname)
#closing_price

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'Date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 985 entries, 2017-03-21 to 2021-02-25
Data columns (total 10 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Value_PHILLIPS 66                        985 non-null    float64
 1   Value_BP P.L.C.                          985 non-null    float64
 2   Value_VALERO ENERGY CORPORATION          985 non-null    float64
 3   Value_CHEVRON CORPORATION                985 non-null    float64
 4   Value_OCCIDENTAL PETROLEUM CORPORATION   985 non-null    float64
 5   Value_MARATHON OIL CORPORATION           985 non-null    float64
 6   Value_PIONEER NATURAL RESOURCES COMPANY  985 non-null    float64
 7   Value_CONOCOPHILLIPS                     985 non-null    float64
 8   Value_EXXON MOBIL CORPORATION            985 non-null    float64
 9   Value_MARATHON PETROLEUM CORP

None

In [164]:
# Load Google Mobility Trends grouped by Indicator
fpath = '../../data/Transportation/google/'
fname = 'baseline_pct_change.csv'
google_mobility = LOAD_DATA(filepath = fpath, filename = fname)
#google_mobility

<class 'pandas.core.indexes.range.RangeIndex'>
TIMESTAMP FOUND! 'date'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 382 entries, 2020-02-15 to 2021-03-02
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   workplaces             382 non-null    float64
 1   retail_and_recreation  382 non-null    float64
 2   grocery_and_pharmacy   382 non-null    float64
 3   residential            382 non-null    float64
 4   transit_stations       382 non-null    float64
 5   parks                  382 non-null    float64
dtypes: float64(6)
memory usage: 20.9 KB


None

<a id='Format-Column-Names'></a>

---
## Format Column Names

[[ go back to the top ]](#Table-of-contents)

In [165]:
# Function to enforce snake case (no spaces, no caps, no DOTs)
def FORMAT_TITLES(dataframe, start = 0, end = 0):
    df = dataframe
    # Remove extra characters in name
    for index, label in enumerate(df.columns):
        # IF not specified, don't remove chars from end
        if end == 0:
            name = label[start:]
        # Remove 'end'-number of chars from end-of-string
        elif end != 0:
            name = label[start:end]
        # Format column names to be in 'snake case'
        formatted_name = name.strip().lower().replace('.', '').replace(" ", "_")
        df.rename(columns = {label:formatted_name}, inplace = True)
    return df

In [166]:
sentiment_int = FORMAT_TITLES(sentiment_int)
sentiment_int.rename(columns = {'daily_news_sentiment':'int_news_sentiment'}, inplace = True)
#sentiment_int

In [167]:
sentiment_na = FORMAT_TITLES(sentiment_na)
sentiment_na.rename(columns = {'daily_news_sentiment':'na_news_sentiment'}, inplace = True)
#sentiment_na

In [168]:
closing_price = FORMAT_TITLES(closing_price, start = 6)
#closing_price

In [169]:
google_mobility = FORMAT_TITLES(google_mobility)
#google_mobility

---
## Set Timestamp Range & Frequency

[[ go back to the top ]](#Table-of-contents)

In [170]:
# Store the latest occurring minimum Timestamp from all the DF's
start = max([df_listed[idx].index[0] for idx in range(len(df_listed))])
start

Timestamp('2020-02-15 00:00:00')

<a id='Merge-Data'></a>

---
## Merge DataFrames

[[ go back to the top ]](#Table-of-contents)

In [171]:
# Identify all DataFrames in this Notebook
%who DataFrame

closing_price	 df	 df_0	 df_1	 df_i	 google_mobility	 sentiment_int	 sentiment_na	 x	 



In [172]:
df_listed = [closing_price, google_mobility, sentiment_int, sentiment_na]
count = 0
for df_i in df_listed:
    # For the first DF, all the others will concat to df_0
    if count == 0:
        # Drop weekend data
        df_0 = df_i[df_i.index.dayofweek < 5]
        # Drop dates before Google Mobililty data begins
        df_0 = df_0.loc[start:].copy()
    else:
        # Drop weekend data
        df_1 = df_i[df_i.index.dayofweek < 5]
        # Concat all the others
        df_0 = pd.concat([df_0, df_1.loc[start:]], axis =1)
    count += 1
display(df_0.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 272 entries, 2020-02-17 to 2021-03-02
Freq: B
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   phillips_66                        244 non-null    float64
 1   bp_plc                             244 non-null    float64
 2   valero_energy_corporation          244 non-null    float64
 3   chevron_corporation                244 non-null    float64
 4   occidental_petroleum_corporation   244 non-null    float64
 5   marathon_oil_corporation           244 non-null    float64
 6   pioneer_natural_resources_company  244 non-null    float64
 7   conocophillips                     244 non-null    float64
 8   exxon_mobil_corporation            244 non-null    float64
 9   marathon_petroleum_corporation     244 non-null    float64
 10  workplaces                         272 non-null    float64
 11  retail_and_recreation          

None

In [173]:
df_0

,phillips_66,bp_plc,valero_energy_corporation,chevron_corporation,occidental_petroleum_corporation,marathon_oil_corporation,pioneer_natural_resources_company,conocophillips,exxon_mobil_corporation,marathon_petroleum_corporation,workplaces,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,int_news_sentiment,na_news_sentiment
date,,,,,,,,,,,,,,,,,,
2020-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-17.756044,4.049697,-0.375113,4.405907,1.903259,28.271605,NaN,NaN
2020-02-18,88.00,36.09,82.52,110.24,41.56,10.30,135.56,58.30,59.88,56.47,-0.006305,-0.211659,-1.377153,1.345745,2.440042,5.466077,NaN,NaN
2020-02-19,88.36,36.25,83.23,110.74,42.51,10.47,141.17,59.26,60.34,57.69,1.001656,2.222899,0.639556,0.345455,2.634615,8.153166,NaN,NaN
2020-02-20,90.19,35.98,84.17,109.81,42.97,10.25,142.53,58.88,59.86,60.26,-0.387448,0.192474,-0.167570,1.922258,2.098260,-1.300847,-0.2,NaN
2020-02-21,89.25,35.36,82.90,109.01,42.12,10.11,142.25,58.44,59.13,59.13,0.547162,2.786383,-1.632662,1.082046,3.656008,6.725381,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,87.25,25.30,78.16,103.31,28.16,11.84,150.06,54.67,56.70,56.65,-18.923493,-6.442705,-2.626381,6.734819,-12.945525,7.219697,-0.1,-0.50
2021-02-25,84.64,25.59,78.15,102.35,26.75,11.30,149.05,53.03,55.76,55.07,-19.029221,-8.706634,-3.693143,7.026578,-13.389535,-2.223188,0.0,0.07
2021-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-18.122059,-11.026122,-7.390536,7.725445,-13.963636,-8.750973,-0.3,-0.40
